# 1. Extracting Text Features

## Imports

In [1]:
import re

import pandas as pd

from razdel import tokenize
from natasha import MorphVocab, Doc, Segmenter, NewsMorphTagger, NewsEmbedding
import tqdm

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kurpenok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Solution

In [2]:
data = pd.read_csv("../data/petitions.csv")

In [3]:
data.drop("id", axis=1, inplace=True)

In [4]:
data["reason_category"].unique()

array(['Благоустройство', 'Содержание МКД',
       'Незаконная информационная и (или) рекламная конструкция', 'Фасад',
       'Водоснабжение', 'Нарушение правил пользования общим имуществом',
       'Повреждения или неисправность элементов уличной инфраструктуры',
       'Кровля', 'Состояние рекламных или информационных конструкций',
       'Нарушение порядка пользования общим имуществом', 'Подвалы',
       'Водоотведение', 'Санитарное состояние', 'Центральное отопление',
       'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)'],
      dtype=object)

In [5]:
data["reason_category"] = data["reason_category"].replace([
    "Повреждения или неисправность элементов уличной инфраструктуры",
    "Состояние рекламных или информационных конструкций",
    "Нарушение порядка пользования общим имуществом",
], "Благоустройство")

data["reason_category"] = data["reason_category"].replace([
    "Фасад",
    "Водоснабжение",
    "Кровля",
    "Подвалы",
    "Водоотведение",
    "Санитарное состояние",
    "Центральное отопление",
], "Содержание МКД")

data["reason_category"] = data["reason_category"].replace([
    "Незаконная информационная и (или) рекламная конструкция",
    "Нарушение правил пользования общим имуществом",
    "Нарушение порядка пользования общим имуществом",
    "Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)",
], "Нарушения")

In [6]:
data["public_petition_text"] = data["public_petition_text"].apply(str.lower)
data["reason_category"] = data["reason_category"].apply(str.lower)

In [7]:
data["public_petition_text"] = data["public_petition_text"].apply(lambda s: re.sub(r"[^\s^\w]+", " ", s))

In [8]:
data["public_petition_text"] = data["public_petition_text"].apply(lambda s: s.replace("\n", ""))

In [9]:
morph_vocab = MorphVocab()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [10]:
def lemmatize(s: str) -> str:
    lemmas = []

    doc = Doc(s)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemmas.append(token.lemma)

    return " ".join(lemmas)

In [11]:
for i, s in enumerate(tqdm.tqdm(data["public_petition_text"], bar_format="[+] Lemmatizing: |{bar:50}|")):
    data["public_petition_text"][i] = lemmatize(s)

[+] Lemmatizing: |██████████████████████████████████████████████████|


In [12]:
stopwords = stopwords.words("russian")

In [13]:
def removing_stop_words(s: str) -> str:
    return [word for word in s.split() if word not in stopwords]

In [14]:
for i, s in enumerate(tqdm.tqdm(data["public_petition_text"], bar_format="[+] Removing stop words: |{bar:50}|")):
    data["public_petition_text"][i] = removing_stop_words(s)

[+] Removing stop words: |██████████████████████████████████████████████████|
